## Análise de Negócios
### Preços dos combustíveis ofertados nos postos

<img src = 'preco_combustivel.jpeg'>

### Fomos contratados por uma Petroleira para construir uma análise de negócios para entender a dinâmica de preços que está sendo ofertado pelos postos.
1 - Teremos que trabalhar com Dados Existentes <br/>
link: https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/serie-historica-de-precos-de-combustiveis <br/>
<img src = 'anp.png'><br/>
<br/>
Como foi uma solicitação do nosso cliente, teremos que avaliar os ultimos 5 anos da base de dados.<br/>
<img src = 'tabela_base_dados.png'><br/>

<br/>
Quando abrimos as tabelas nos deparamos que cada semestre possui uma quantidade muito grande de dados, logo juntas 5 anos (10 tabelas) fica difícil!<br/>
<br/>
Como muitas das vezes não temos o Engenheiro de Dados, temos que dar os nossos pulos!<br/>


2 - Armazenar esses dados em um banco de dados <br/>
<img src = 'PostgreSQL-logo.png'><br/>
<img src = 'postgre1.png'><br/>

In [1]:
"""
CREATE TABLE anp.preco_combustivel(
		regiao 				varchar(255)
		,estado				varchar(255)
		,municipio			varchar(255)
		,revenda			varchar(255)
		,cnpj				varchar(255)
		,nome_rua			varchar(255)	
		,numero_rua			varchar(255)
		,complemento		varchar(255)
		,bairro				varchar(255)
		,cep				varchar(255)
		,produto			varchar(255)
		,data_coleta		date
		,valor_venda		float
		,unidade_medida		varchar(255)
		,bandeira			varchar(255)

)
"""

'\nCREATE TABLE anp.preco_combustivel(\n\t\tregiao \t\t\t\tvarchar(255)\n\t\t,estado\t\t\t\tvarchar(255)\n\t\t,municipio\t\t\tvarchar(255)\n\t\t,revenda\t\t\tvarchar(255)\n\t\t,cnpj\t\t\t\tvarchar(255)\n\t\t,nome_rua\t\t\tvarchar(255)\t\n\t\t,numero_rua\t\t\tvarchar(255)\n\t\t,complemento\t\tvarchar(255)\n\t\t,bairro\t\t\t\tvarchar(255)\n\t\t,cep\t\t\t\tvarchar(255)\n\t\t,produto\t\t\tvarchar(255)\n\t\t,data_coleta\t\tdate\n\t\t,valor_venda\t\tfloat\n\t\t,unidade_medida\t\tvarchar(255)\n\t\t,bandeira\t\t\tvarchar(255)\n\n)\n'

Carga de dados no Banco! <br/>
Utilizaremos o Knime<br/>
<img src = 'knime.png'><br/>
<img src = 'knime2.png'><br/>
Para atender as nossas necessidades vamos construir desta forma.<br/>
<img src = 'knime3.png'><br/>

3 - Utilizar esses dados para as nossas análises <br/>

Instalando as bibliotecas

In [2]:
#pip install psycopg2

In [3]:
import pandas as pd
import pandas.io.sql as sqlio
import psycopg2 as ps

import warnings
warnings.filterwarnings('ignore')

In [4]:
conn = ps.connect(dbname = 'anp',
                    user ='postgres',
                    password = 'degard',
                    host = 'localhost',
                    port = '5432' )

In [5]:
sql = """
SELECT * FROM anp.preco_combustivel
"""

In [6]:
df = sqlio.read_sql_query(sql, conn)

Visualizar as 5 primeiras informações

In [7]:
df.head()

,regiao,estado,municipio,revenda,cnpj,nome_rua,numero_rua,complemento,bairro,cep,produto,data_coleta,valor_venda,unidade_medida,bandeira
0,SE,MG,ARAGUARI,AUTO POSTO JL LTDA,06.315.263/0001-76,AVENIDA VEREADOR GERALDO TEODORO,290,None,BOSQUE,38446-124,DIESEL S10,2022-07-26,7.69,R$ / litro,VIBRA ENERGIA
1,SE,MG,ARAGUARI,AUTO POSTO JL LTDA,06.315.263/0001-76,AVENIDA VEREADOR GERALDO TEODORO,290,None,BOSQUE,38446-124,DIESEL,2022-07-26,7.69,R$ / litro,VIBRA ENERGIA
2,SE,MG,ARAGUARI,AUTO POSTO JL LTDA,06.315.263/0001-76,AVENIDA VEREADOR GERALDO TEODORO,290,None,BOSQUE,38446-124,ETANOL,2022-07-26,4.09,R$ / litro,VIBRA ENERGIA
3,SE,MG,ARAGUARI,AUTO POSTO JL LTDA,06.315.263/0001-76,AVENIDA VEREADOR GERALDO TEODORO,290,None,BOSQUE,38446-124,GASOLINA,2022-07-26,5.59,R$ / litro,VIBRA ENERGIA
4,SE,MG,OLIVEIRA,POSTO PERFORMANCE LTDA.,26.391.185/0001-04,RUA AMINTAS DE SOUZA,149,None,CENTRO,35540-000,DIESEL,2022-07-26,7.25,R$ / litro,BRANCA


In [8]:
print(f'Quantidade de linhas e colunas {df.shape}')

Quantidade de linhas e colunas (4401370, 15)


Tipo das Colunas

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4401370 entries, 0 to 4401369
Data columns (total 15 columns):
 #   Column          Dtype  
---  ------          -----  
 0   regiao          object 
 1   estado          object 
 2   municipio       object 
 3   revenda         object 
 4   cnpj            object 
 5   nome_rua        object 
 6   numero_rua      object 
 7   complemento     object 
 8   bairro          object 
 9   cep             object 
 10  produto         object 
 11  data_coleta     object 
 12  valor_venda     float64
 13  unidade_medida  object 
 14  bandeira        object 
dtypes: float64(1), object(14)
memory usage: 503.7+ MB


In [10]:
df['data_coleta']= pd.to_datetime(df['data_coleta'])

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4401370 entries, 0 to 4401369
Data columns (total 15 columns):
 #   Column          Dtype         
---  ------          -----         
 0   regiao          object        
 1   estado          object        
 2   municipio       object        
 3   revenda         object        
 4   cnpj            object        
 5   nome_rua        object        
 6   numero_rua      object        
 7   complemento     object        
 8   bairro          object        
 9   cep             object        
 10  produto         object        
 11  data_coleta     datetime64[ns]
 12  valor_venda     float64       
 13  unidade_medida  object        
 14  bandeira        object        
dtypes: datetime64[ns](1), float64(1), object(13)
memory usage: 503.7+ MB


Verificar se há nulos nas colunas

In [12]:
df.isnull().sum()

regiao                  0
estado                  0
municipio               0
revenda                 0
cnpj                    0
nome_rua                0
numero_rua           2775
complemento       3354329
bairro              13416
cep                     0
produto                 0
data_coleta             0
valor_venda             0
unidade_medida          0
bandeira                0
dtype: int64

Selecionar as colunas que vamos trabalhar<br/>
regiao, estado, municipio, produto, data_coleta, valor_venda, bandeira

In [13]:
df_anp = df[['data_coleta', 'regiao', 'estado', 'municipio', 'bandeira','produto', 'valor_venda']]

In [14]:
df_anp.head()

,data_coleta,regiao,estado,municipio,bandeira,produto,valor_venda
0,2022-07-26,SE,MG,ARAGUARI,VIBRA ENERGIA,DIESEL S10,7.69
1,2022-07-26,SE,MG,ARAGUARI,VIBRA ENERGIA,DIESEL,7.69
2,2022-07-26,SE,MG,ARAGUARI,VIBRA ENERGIA,ETANOL,4.09
3,2022-07-26,SE,MG,ARAGUARI,VIBRA ENERGIA,GASOLINA,5.59
4,2022-07-26,SE,MG,OLIVEIRA,BRANCA,DIESEL,7.25


In [15]:
df_anp['ano'] = df_anp['data_coleta'].dt.year
df_anp['mes'] = df_anp['data_coleta'].dt.month

In [16]:
df_anp.head()

,data_coleta,regiao,estado,municipio,bandeira,produto,valor_venda,ano,mes
0,2022-07-26,SE,MG,ARAGUARI,VIBRA ENERGIA,DIESEL S10,7.69,2022,7
1,2022-07-26,SE,MG,ARAGUARI,VIBRA ENERGIA,DIESEL,7.69,2022,7
2,2022-07-26,SE,MG,ARAGUARI,VIBRA ENERGIA,ETANOL,4.09,2022,7
3,2022-07-26,SE,MG,ARAGUARI,VIBRA ENERGIA,GASOLINA,5.59,2022,7
4,2022-07-26,SE,MG,OLIVEIRA,BRANCA,DIESEL,7.25,2022,7


Estatística Básica

In [17]:
df_anp.describe().round(2)

,data_coleta,valor_venda,ano,mes
count,4401370,4401370.00,4401370.00,4401370.00
mean,2020-05-29 04:40:07.617630464,4.48,2019.93,6.27
min,2018-01-01 00:00:00,1.76,2018.00,1.00
25%,2019-02-18 00:00:00,3.52,2019.00,3.00
50%,2020-03-25 00:00:00,4.19,2020.00,6.00
75%,2021-10-20 00:00:00,5.20,2021.00,9.00
max,2022-12-30 00:00:00,9.65,2022.00,12.00
std,NaN,1.30,1.45,3.38


Quais são os tipos de produtos que são comercializados?

In [18]:
#Visualizando os valores únicos
print(f'Os produtos conecializados{df_anp.produto.unique()}')

Os produtos conecializados['DIESEL S10' 'DIESEL' 'ETANOL' 'GASOLINA' 'GASOLINA ADITIVADA' 'GNV']


Quais anos estão na Base?

In [19]:
#Visualizando os valores únicos
print(f'Os anos da Base: {df_anp.ano.unique()}')

Os anos da Base: [2022 2018 2019 2020 2021]


Descobrindo os valores mínimos, máximos e médios dos produtos por ano

In [29]:
df_anp_valor = df_anp[['ano', 'produto', 'valor_venda']]

In [30]:
df_anp_valor.groupby(['produto','ano']).agg(['min', 'max', 'mean']).round(2)

valor_venda            
                                min   max  mean
produto            ano                         
DIESEL             2018        2.80  5.20  3.50
                   2019        2.87  4.99  3.60
                   2020        2.45  5.09  3.42
                   2021        3.10  6.99  4.69
                   2022        3.14  9.00  6.63
DIESEL S10         2018        2.84  5.30  3.59
                   2019        2.79  5.09  3.69
                   2020        2.46  5.38  3.51
                   2021        2.80  6.96  4.74
                   2022        3.59  9.65  6.73
ETANOL             2018        1.90  4.98  3.13
                   2019        2.10  5.47  3.17
                   2020        1.80  5.15  3.18
                   2021        2.05  7.90  4.67
                   2022        2.49  7.98  4.85
GASOLINA           2018        3.37  6.29  4.45
                   2019        3.39  6.29  4.42
                   2020        2.87  5.90  4.28
                   2021        3.10  8.00  5.89
                   2022        3.49  8.99  6.32
GASOLINA ADITIVADA 2020        3.69  6.00  4.59
                   2021        3.46  8.99  6.04
                   2022        4.09  9.28  6.47
GNV                2018        1.76  4.10  2.73
                   2019        2.00  4.56  3.22
                   2020        2.00  4.78  3.17
                   2021        2.38  6.70  3.89
                   2022        3.17  7.99  5.04

In [31]:
df_anp_valor_estado = df_anp[['ano', 'produto', 'estado','valor_venda']]

In [32]:
df_anp_valor_estado.groupby(['produto','ano','estado']).agg(['min', 'max', 'mean']).round(2)

valor_venda            
                            min   max  mean
produto ano  estado                        
DIESEL  2018 AC            3.38  4.89  4.29
             AL            3.10  4.23  3.60
             AM            3.09  4.49  3.63
             AP            3.29  4.51  3.88
             BA            2.99  5.20  3.51
...                         ...   ...   ...
GNV     2022 RN            4.11  4.99  4.50
             RS            4.59  6.89  5.88
             SC            4.00  6.89  5.50
             SE            4.47  5.49  4.99
             SP            3.59  6.99  5.22

[727 rows x 3 columns]

Descobrir a quantidade de bandeiras por produto/ano

In [22]:
df_anp_bandeira = df_anp[['ano','bandeira','produto']]

In [25]:
df_anp_bandeira.groupby(['ano','produto']).bandeira.nunique()

ano   produto           
2018  DIESEL                64
      DIESEL S10            66
      ETANOL                66
      GASOLINA              66
      GNV                   24
2019  DIESEL                59
      DIESEL S10            61
      ETANOL                63
      GASOLINA              64
      GNV                   22
2020  DIESEL                52
      DIESEL S10            57
      ETANOL                58
      GASOLINA              58
      GASOLINA ADITIVADA    43
      GNV                   23
2021  DIESEL                54
      DIESEL S10            54
      ETANOL                55
      GASOLINA              55
      GASOLINA ADITIVADA    54
      GNV                   22
2022  DIESEL                51
      DIESEL S10            60
      ETANOL                60
      GASOLINA              60
      GASOLINA ADITIVADA    58
      GNV                   24
Name: bandeira, dtype: int64

In [34]:
df_anp_bandeira_estado = df_anp[['ano','bandeira','produto', 'estado']]

In [35]:
df_anp_bandeira_estado.groupby(['ano','produto','estado']).bandeira.nunique()

ano   produto  estado
2018  DIESEL   AC         6
               AL         7
               AM         9
               AP         3
               BA        10
                         ..
2022  GNV      RN         5
               RS         6
               SC         8
               SE         5
               SP         7
Name: bandeira, Length: 727, dtype: int64